In [1]:
%load_ext tensorboard

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import datetime

2025-01-29 22:41:49.964782: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-29 22:41:50.083637: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-29 22:41:50.083721: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-29 22:41:50.093529: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-29 22:41:50.126433: I tensorflow/core/platform/cpu_feature_guar

In [3]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import ActivityRegularization, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Activation

In [4]:
# gpu settings

gpus = tf.config.list_physical_devices("GPU")

if gpus:
    print("GPU detected: ", gpus[0])
    try:
        tf.config.set_visible_devices(gpus[0], "GPU")
        ldev = tf.config.list_logical_devices("GPU")
        print(len(ldev), "Logical devices and ", len(gpus), "Physical devices detected")
    except RuntimeError as e:
        print(e)

if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except:
        pass

os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"
print(os.getenv("TF_GPU_ALLOCATOR"))

tf.debugging.set_log_device_placement(False)

GPU detected:  PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
1 Logical devices and  1 Physical devices detected
cuda_malloc_async


2025-01-29 22:41:52.768997: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-01-29 22:41:52.866407: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-01-29 22:41:52.866582: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [5]:
train_dir = "/home/abhinav/Documents/Work/2 Hobby_projects/Datasets/Beetles_dataset/tvt_final/train"
valid_dir = "/home/abhinav/Documents/Work/2 Hobby_projects/Datasets/Beetles_dataset/tvt_final/test"
test_dir = "/home/abhinav/Documents/Work/2 Hobby_projects/Datasets/Beetles_dataset/tvt_final/val"

model_dir = "/home/abhinav/Documents/Work/2 Hobby_projects/Models"

In [6]:
# # To split a folder containing dataset into train, valid, test folders
# # Just mention the folder containing sub folders each having a specific image related to folder
# # This would create train, val, test folders, each having same file structure

# caltech101_dir = "/home/abhinav/Documents/Work/2 Hobby_projects/Datasets/Beetles_dataset/tvt"
# caltech101_out = "/home/abhinav/Documents/Work/2 Hobby_projects/Datasets/Beetles_dataset/tvt_final"

# import splitfolders
# splitfolders.ratio(caltech101_dir, output=caltech101_out, seed=1337, ratio=(.5, 0.3,0.2))   #Train, Valid, Test

In [7]:
num_epochs = 7
batch_size = 4
img_h = 140
img_w = 140# To split a folder containing dataset into train, valid, test folders
# # Just mention the folder containing sub folders each having a specific image related to folder
# # This would create train, val, test folders, each having same file structure

# caltech101_dir = "/home/abhinav/Documents/Work/2 Hobby_projects/Datasets/Beetles_dataset/tvt"
# caltech101_out = "/home/abhinav/Documents/Work/2 Hobby_projects/Datasets/Beetles_dataset/tvt_final"

# import splitfolders
# splitfolders.ratio(caltech101_dir, output=caltech101_out, seed=1337, ratio=(.5, 0.3,0.2))   #Train, Valid, Test
num_channels = 3
num_classes = 4

In [8]:
# # ACcuracy 77

# class MyNet(Sequential):
#     def __init__(self, input_shape, num_classes):
#         super().__init__()

#         self.add(Conv2D(
#             512,
#             kernel_size=(3, 3),
#             strides=1,
#             padding="valid",
#             kernel_initializer="he_normal",
#             input_shape=input_shape,
#             activation="relu"
#         ))

#         self.add(Conv2D(
#             256,
#             kernel_size=(3, 3),
#             strides=2,
#             padding="valid",
#             kernel_initializer="he_normal",
#             activation="relu"
#         ))
        
#         self.add(MaxPooling2D(
#             pool_size=(3, 3),
#             strides=2,
#             padding="valid",
#             data_format=None
#         ))

#         self.add(Conv2D(
#             256,
#             kernel_size=(3, 3),
#             strides=1,
#             padding="valid",
#             kernel_initializer="he_normal",
#             activation="relu"
#         ))

#         self.add(Conv2D(
#             256,
#             kernel_size=(3, 3),
#             strides=1,
#             padding="valid",
#             kernel_initializer="he_normal",
#             activation="relu"
#         ))

#         self.add(MaxPooling2D(
#             pool_size=(3, 3),
#             strides=2,
#             padding="valid",
#             data_format=None
#         ))

#         self.add(Conv2D(
#             256,
#             kernel_size=(3, 3),
#             strides=1,
#             padding="valid",
#             kernel_initializer="he_normal",
#             activation="relu"
#         ))

#         self.add(Conv2D(
#             256,
#             kernel_size=(3, 3),
#             strides=2,
#             padding="valid",
#             kernel_initializer="he_normal",
#             activation="relu"
#         ))

#         self.add(Conv2D(
#             512,
#             kernel_size=(3, 3),
#             strides=2,
#             padding="valid",
#             kernel_initializer="he_normal",
#             activation="relu"
#         ))

#         # self.add(BatchNormalization(momentum=1))
#         self.add(Flatten())
#         # self.add(Dropout(0.5))
#         # self.add(Dense(4096, activation="relu"))
#         self.add(Dropout(0.05))
#         # self.add(Dense(3796, activation="relu"))
#         self.add(Dense(1350, activation="relu"))
#         self.add(Dense(num_classes, activation="softmax"))

#         self.compile(
#             # optimizer=Ranger(learning_rate=0.0001),
#             # optimizer=Ranger(learning_rate=0.000001, amsgrad=True, slow_step_size=0.8, warmup_proportion=0.3),
#             optimizer=Adam(learning_rate=0.0001, use_ema=True, ema_momentum=1),
#             loss="categorical_crossentropy",
#             metrics=["accuracy"]
#         )


In [9]:
# ACcuracy 77

class MyNet(Sequential):
    def __init__(self, input_shape, num_classes):
        super().__init__()

        self.add(Conv2D(
            512,
            kernel_size=(3, 3),
            strides=1,
            padding="valid",
            kernel_initializer="he_normal",
            input_shape=input_shape,
            activation="relu"
        ))

        self.add(Conv2D(
            512,
            kernel_size=(3, 3),
            strides=1,
            padding="valid",
            kernel_initializer="he_normal",
            activation="relu"
        ))
        
        self.add(MaxPooling2D(
            pool_size=(3, 3),
            strides=2,
            padding="valid",
            data_format=None
        ))

        self.add(Conv2D(
            256,
            kernel_size=(3, 3),
            strides=2,
            padding="valid",
            kernel_initializer="he_normal",
            activation="relu"
        ))

        self.add(Conv2D(
            256,
            kernel_size=(3, 3),
            strides=2,
            padding="valid",
            kernel_initializer="he_normal",
            activation="relu"
        ))

        self.add(MaxPooling2D(
            pool_size=(3, 3),
            strides=2,
            padding="valid",
            data_format=None
        ))

        self.add(Conv2D(
            128,
            kernel_size=(3, 3),
            strides=1,
            padding="valid",
            kernel_initializer="he_normal",
            activation="relu"
        ))

        self.add(Conv2D(
            128,
            kernel_size=(3, 3),
            strides=1,
            padding="valid",
            kernel_initializer="he_normal",
            activation="relu"
        ))

        self.add(Conv2D(
            128,
            kernel_size=(3, 3),
            strides=2,
            padding="valid",
            kernel_initializer="he_normal",
            activation="relu"
        ))

        # self.add(BatchNormalization(momentum=1))
        self.add(Flatten())
        # self.add(Dropout(0.5))
        # self.add(Dense(4096, activation="relu"))
        self.add(Dropout(0.18))
        # self.add(Dense(3796, activation="relu"))
        self.add(Dense(1400, activation="relu"))
        self.add(Dense(num_classes, activation="softmax"))

        self.compile(
            # optimizer=Ranger(learning_rate=0.0001),
            # optimizer=Ranger(learning_rate=0.000001, amsgrad=True, slow_step_size=0.8, warmup_proportion=0.3),
            optimizer=Adam(learning_rate=0.0001, use_ema=True, ema_momentum=1),
            loss="categorical_crossentropy",
            metrics=["accuracy"]
        )


In [10]:
# model = tf.keras.applications.mobilenet_v2.MobileNetV2()

# model.outputs = [model.layers[-1].output]
# x=Dense(num_classes, activation='softmax')(model.output)
# model=Model(model.input,x) #To set the first 8 layers to non-trainable (weights will not be updated)

# # model.add(Dense(units=2))
# # model.add(Activation('softmax'))
# model.compile(
#             # optimizer=Ranger(learning_rate=0.0001),
#             # optimizer=Ranger(learning_rate=0.000001, amsgrad=True, slow_step_size=0.8, warmup_proportion=0.3),
#             optimizer=Adam(learning_rate=0.0001, use_ema=True, ema_momentum=1),
#             loss="categorical_crossentropy",
#             metrics=["accuracy"]
#         )
# model.summary()

In [11]:
# model = tf.keras.applications.efficientnet_v2.EfficientNetV2S()

# model.outputs = [model.layers[-1].output]
# x=Dense(num_classes, activation='softmax')(model.output)
# model=Model(model.input,x) #To set the first 8 layers to non-trainable (weights will not be updated)

# # model.add(Dense(units=2))
# # model.add(Activation('softmax'))
# model.compile(
#             # optimizer=Ranger(learning_rate=0.0001),
#             # optimizer=Ranger(learning_rate=0.000001, amsgrad=True, slow_step_size=0.8, warmup_proportion=0.3),
#             optimizer=Adam(learning_rate=0.0001, use_ema=True, ema_momentum=1),
#             loss="categorical_crossentropy",
#             metrics=["accuracy"]
#         )
# model.summary()

In [12]:
# model = tf.keras.applications.vgg19.VGG19()

# model.outputs = [model.layers[-1].output]
# x=Dense(num_classes, activation='softmax')(model.output)
# model=Model(model.input,x) #To set the first 8 layers to non-trainable (weights will not be updated)

# # model.add(Dense(units=2))
# # model.add(Activation('softmax'))
# model.compile(
#             # optimizer=Ranger(learning_rate=0.0001),
#             # optimizer=Ranger(learning_rate=0.000001, amsgrad=True, slow_step_size=0.8, warmup_proportion=0.3),
#             optimizer=Adam(learning_rate=0.0001, use_ema=True, ema_momentum=1),
#             loss="categorical_crossentropy",
#             metrics=["accuracy"]
#         )
# model.summary()

In [13]:
# model = tf.keras.applications.efficientnet.EfficientNetB2()

# model.outputs = [model.layers[-1].output]
# x=Dense(num_classes, activation='softmax')(model.output)
# model=Model(model.input,x) #To set the first 8 layers to non-trainable (weights will not be updated)

# # model.add(Dense(units=2))
# # model.add(Activation('softmax'))
# model.compile(
#             # optimizer=Ranger(learning_rate=0.0001),
#             # optimizer=Ranger(learning_rate=0.000001, amsgrad=True, slow_step_size=0.8, warmup_proportion=0.3),
#             optimizer=Adam(learning_rate=0.0001, use_ema=True, ema_momentum=1),
#             loss="categorical_crossentropy",
#             metrics=["accuracy"]
#         )
# model.summary()

In [14]:
# model = tf.keras.applications.mobilenet_v2.MobileNetV2()

# model.outputs = [model.layers[-1].output]
# x=Dense(num_classes, activation='softmax')(model.output)
# model=Model(model.input,x) #To set the first 8 layers to non-trainable (weights will not be updated)

# # model.add(Dense(units=2))
# # model.add(Activation('softmax'))
# model.compile(
#             # optimizer=Ranger(learning_rate=0.0001),
#             # optimizer=Ranger(learning_rate=0.000001, amsgrad=True, slow_step_size=0.8, warmup_proportion=0.3),
#             optimizer=Adam(learning_rate=0.0001, use_ema=True, ema_momentum=1),
#             loss="categorical_crossentropy",
#             metrics=["accuracy"]
#         )
# model.summary()

In [15]:
# model = load_model(os.path.join(model_dir, "Mynet1"))
# model.summary()

In [16]:
# model = load_model(os.path.join(model_dir, "AlexNet"))
# model.summary()

In [17]:
model = MyNet((img_w, img_h, num_channels), num_classes)
model.summary()

Model: "my_net"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 138, 138, 512)     14336     
                                                                 
 conv2d_1 (Conv2D)           (None, 136, 136, 512)     2359808   
                                                                 
 max_pooling2d (MaxPooling2  (None, 67, 67, 512)       0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 33, 33, 256)       1179904   
                                                                 
 conv2d_3 (Conv2D)           (None, 16, 16, 256)       590080    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 7, 7, 256)         0         
 g2D)                                                       

In [18]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=10,
    width_shift_range=1,
    height_shift_range=1,
    shear_range=0.6,
    zoom_range=1
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_w, img_h),
    color_mode="rgb",
    batch_size=batch_size,
    seed=1,
    shuffle=True,
    class_mode="categorical"
)

valid_datagen = ImageDataGenerator(
    rescale=1./255
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(img_w, img_h),
    color_mode="rgb",
    batch_size=batch_size,
    seed=7,
    shuffle=True,
    class_mode="categorical"
)

test_datagen = ImageDataGenerator(
    rescale=1.0/255.0
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_w, img_h),
    color_mode="rgb",
    batch_size=batch_size,
    seed=7,
    shuffle=True,
    class_mode="categorical"
)

train_num = train_generator.samples
valid_num = valid_generator.samples
train_num = train_generator.samples

Found 3133 images belonging to 4 classes.
Found 1258 images belonging to 4 classes.
Found 1878 images belonging to 4 classes.


In [19]:
logs_dir = "./logs/fit" + datetime.datetime.now().strftime("%d%M%Y-%H%M%S")
tensorboard_callbacks = tf.keras.callbacks.TensorBoard(log_dir=logs_dir)
callbacks_list = [tensorboard_callbacks]

model.fit(
    train_generator,
    epochs=9,
    batch_size=batch_size,
    validation_data=valid_generator,
    callbacks=callbacks_list,
    verbose=1
)

2025-01-29 22:41:53.975722: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Epoch 1/9


2025-01-29 22:41:55.608449: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2025-01-29 22:41:55.709722: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2025-01-29 22:42:02.498642: I external/local_xla/xla/service/service.cc:168] XLA service 0x718314fdc140 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-01-29 22:42:02.498668: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2025-01-29 22:42:02.504203: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1738170722.606101   55357 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2025-01-29 22:42:06.664649: W extern

784/784 [==============================] - 271s 313ms/step - loss: 1.0464 - accuracy: 0.5464 - val_loss: 0.7417 - val_accuracy: 0.6741
Epoch 2/9
784/784 [==============================] - 250s 318ms/step - loss: 0.7994 - accuracy: 0.6696 - val_loss: 0.6557 - val_accuracy: 0.7273
Epoch 3/9
562/784 [====================>.........] - ETA: 58s - loss: 0.7244 - accuracy: 0.7007

KeyboardInterrupt: 

In [20]:
model.evaluate(
    test_generator,
    verbose=2
)

470/470 - 49s - loss: 0.6245 - accuracy: 0.7433 - 49s/epoch - 104ms/step


[0.6245176196098328, 0.7433440089225769]

In [21]:
model.save(os.path.join(model_dir, "Mynet_beetles_7"))

INFO:tensorflow:Assets written to: /home/abhinav/Documents/Work/2 Hobby_projects/Models/Mynet_beetles_7/assets


INFO:tensorflow:Assets written to: /home/abhinav/Documents/Work/2 Hobby_projects/Models/Mynet_beetles_7/assets
